# 01. 순환 신경망 (RNN & LSTM)
> 순환신경망 (RNN)에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 7]
- permalink: /rnn_lstm
- exec: colab

### 1. RNN as Auto Regressive Model

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day5/reg05.png?raw=true)
 
우리는 이미 회귀모델 시간에 '자기 종속성'이라는 속성에 대해 이야기 한적이 있습니다. 자기종속성이란 이전 타임스텝의 출력이 다음 타임스텝의 입력에 영향을 준다는 속성입니다. 우리가 오늘 배울 RNN이라는 신경망은 바로 이러한 자기 종속성을 다층 퍼셉트론으로 모델링한 구조입니다. <br><br>

음악, 동영상, 에세이, 시, 소스 코드, 주가 차트. 이것들의 공통점은 무엇일까요? 바로 **시간적인 순서를 갖는 시퀀스**라는 점입니다. 음악은 음계들의 시퀀스, 동영상은 이미지의 시퀀스, 에세이는 단어들의 시퀀스로 볼 수 있습니다. 시퀀스의 길이는 가변적입니다. 소설에는 한 페이지짜리 단편소설도 있고 열권짜리 장편소설도 있죠. 기존의 뉴럴 네트워크 알고리즘은 고정된 크기의 입력을 다루는 데는 탁월하지만, 가변적인 크기의 데이터를 모델링하기에는 적합하지 않습니다. <br><br>

RNN(Recurrent Neural Network, 순환신경망)은 **시퀀스 데이터를 모델링 하기 위해 등장**했습니다. RNN이 기존의 뉴럴 네트워크와 다른 점은 **‘기억’(다른 말로 hidden state)을 갖고 있다는 점**입니다. 네트워크의 기억은 지금까지의 입력 데이터를 요약한 정보라고 볼 수 있습니다. 새로운 입력이 들어올때마다 네트워크는 자신의 기억을 조금씩 수정합니다. 결국 입력을 모두 처리하고 난 후 네트워크에게 남겨진 기억은 시퀀스 전체를 요약하는 정보가 됩니다. 이는 사람이 시퀀스를 처리하는 방식과 비슷합니다. 이 글을 읽을 때도 우리는 이전까지의 단어에 대한 기억을 바탕으로 새로운 단어를 이해합니다. <br><br>

이 과정은 새로운 단어마다 계속해서 반복되기 때문에 RNN에는 Recurrent, 즉 순환적이라는 이름이 붙습니다. RNN은 이런 반복을 통해 아무리 긴 시퀀스라도 처리할 수 있는 것입니다. <br><br>

![](https://files.slack.com/files-pri/T25783BPY-F6YUKQKCP/rnn-diagram.png?pub_secret=9e9b7d3f1e)

위 다이어그램에서 빨간색 사각형은 입력, 노란색 사각형은 기억, 파란색 사각형은 출력을 나타냅니다. 첫번째 입력이 들어오면 첫번째 기억이 만들어집니다. 두번째 입력이 들어오면 기존의 기억과 새로운 입력을 참고하여 새 기억을 만듭니다. 입력의 길이만큼 이 과정을 얼마든지 반복할 수 있습니다. 각각의 기억은 그때까지의 입력을 요약해서 갖고 있는 정보입니다. RNN은 이 요약된 정보를 바탕으로 출력을 만들어 냅니다. <br><br>

### 2. What can RNNs do?

RNN의 입력과 출력은 우리가 네트워크에게 시키고 싶은 것이 무엇이냐에 따라 얼마든지 달라질 수 있습니다. 아래는 몇가지 예시입니다. <br><br>

#### 2.1. 시계열 예측(e.g. 주가 예측)
![](https://t1.daumcdn.net/cfile/tistory/99C2D5465C66628B13)

주가예측과 같은 시계열 예측 문제를 RNN으로 풀어볼 수 있습니다. 주가예측의 경우 이전 시간의 주가가 다음 시간의 주가에 영향을 주기 때문에 Auto regressive model인 RNN을 사용해서 문제를 풀어볼 수 있는 것입니다. 
<br><br>

#### 2.2. 자연어 처리(e.g. 기계 번역)

![](https://files.slack.com/files-pri/T25783BPY-F6X6W5UTA/machine-translator.png?pub_secret=c4b0c11754)

구글의 번역기와 네이버의 파파고와 같은 자연어처리 시스템들은 RNN을 응용한 모델로 만들어졌습니다. RNN 기반 모델은 기존 통계 기반 모델의 비해 우수한 성능을 낸다고 알려져 있습니다. 자연어 역시 마찬가지로 이전 단어에 의해 다음 단어가 결정되므로 RNN을 사용할 수 있는 것입니다. <br><br>

#### 2.3. 음성 처리(e.g. 음성 인식)

![](https://file.hstatic.net/1000400311/file/chuyen_van_ban_thanh_giong_noi_c72b34f374f646e7891357808323eee2.jpg)

대부분의 음성처리 시스템들도 RNN을 기반으로 만들어진 것으로 알려졌습니다. 소리 역시 이전 소리가 다음 소리에 영향을 주기 때문에, 이러한 관계를 모델링하기 위해 RNN을 사용합니다.
<br><br>

### 3. More Details in RNN

![](https://t1.daumcdn.net/cfile/tistory/261D7C4757AC186F04)

RNN은 sequence data를 처리하는 모델입니다. sequence는 순서대로 처리해야 하는 것을 뜻하고, 이런 데이터에는 시계열 처리, 음성인식, 자연어처리 등이 포함됩니다. 자연어의 경우 단어 하나만 안다고 해서 처리될 수 없고, 앞뒤 문맥을 함께 이해해야 해석이 가능하기 때문입니다. 이러한 작업은 적어도, 지금까지 열심히 공부한 neural network나 다양한 머신러닝 모델들, 앞으로 배울 CNN으로도 이렇게 시간적인 특성을 가르치는 것은 불가능합니다. <br><br>

![](https://t1.daumcdn.net/cfile/tistory/2578204757AC186F2A)

위 그림에서 보면 $x_t$가 입력되어서 히든레이어 $A$를 거쳐서 최종적으로 $h_t$라는 출력을 반환하게 됩니다. 이것만 보면 이전에 배운 다층 퍼셉트론(MLP)와 차이가 없습니다. 그러나 RNN의 특징은 저기에 있는 순환구조의 화살표에 있습니다. 저 화살표는 히든레이어인 $A$에서 나와서 다시 히든레이어 $A$로 들어갑니다. 이 것을 간단하게 표현하기 위해 시간순으로 펼쳐서(unfold) 오른쪽 처럼도 표현할 수 있습니다. 이렇게 보면 시간의 흐름에 따라 같은 모양의 신경망이 마치 복사된 것처럼 느껴집니다
<br><br>

![](https://t1.daumcdn.net/cfile/tistory/2201424757AC187023)

RNN(Recurrent Neural Network)은 일반적으로 step을 거칠 때마다 어떤 결과를 예측하게 됩니다. 그리고, 이런 예측 값을 앞에서 배웠던 것처럼 y라고 부릅니다. y = Wx + b와 같이 생각하시면 됩니다. 그리고 RNN에서는 y 대신에 h를 자주 사용하는 편입니다. 따라서 저도 h로 놓고 설명드리도록 하겠습니다.
<br><br>

![](https://t1.daumcdn.net/cfile/tistory/271F9B4757AC187002)

수식으로 표현하면 위와 같습니다. 새로운 출력인 $h_t$는 이전의 출력인 $h_{t-1}$과 새로운 입력 $x_t$이 입력되어 나온 출력입니다.  여기서 노드 갯수는 layer에 포함된 노드(그림에서는 초록색으로 표시된 RNN) 갯수를 말합니다. <br><br>

![](https://t1.daumcdn.net/cfile/tistory/21140B4757AC18710F)

수식을 좀 더 자세히 보겠습니다. 오른쪽 그림에서 가장 먼저 $x_0$이 입력됩니다. 그러면 히든레이어 $A$를 거쳐서 $h_0$이 출력됩니다. 그리고 나서 그 다음 타임스텝의 사건이 $x_1$으로 입력됩니다. 그러면 RNN 모델은 현재 타임스텝의 입력인 $x_1$과 이전 타임스텝까지 저장하고 있던 출력$h_0$을 다시 히든레이어에 입력합니다. 그래서 이 둘이 더해져서 현재 타임스텝인 $h_1$가 완성됩니다. 이를 수식으로 표현하면 아래와 같습니다.

<br>
$$h_1 = activation(w_x \cdot x_1 + w_h \cdot h_0)$$
<br><br>

![](https://t1.daumcdn.net/cfile/tistory/274D224F57AC187225)

이 때, $x$와 $h$가 입력될 때, 각각 $w_x$와 $w_h$가 곱해지고나서 더해지는데, 이들이 바로 RNN의 가중치입니다. 한가지 주의할 점은 매 타임스텝마다 다른 $w$들이 있는 것이 아니라, $w$는 $w_x$와 $w_h$로 딱 2개만 존재하고, 매 타임스텝마다 들어오는 입력들이 매번 같은 weight와 곱해진다는 것입니다. 


<br>

### 4. RNN으로 시계열 예측(주가 예측)을 해보자.

백문이 불여 일견이죠. RNN에 대해 어느정도 이해하셨다면 코드로 구현해봅시다. 우리는 여기에서 시계열 예측 문제 중 하나인 주가 예측 문제를 풀어보려고합니다. 

In [3]:
import pandas as pd

df_price = pd.read_csv('sample_data/samsung.csv', encoding='utf8')
df_price

,일자,시가,고가,저가,종가,거래량
0,20200107,55700,56400,55600,55800,9893846
1,20200106,54900,55600,54600,55500,10278951
2,20200103,56000,56600,54900,55500,15422255
3,20200102,55500,56000,55000,55200,12993228
4,20191230,56200,56600,55700,55800,8356767
...,...,...,...,...,...,...
9283,19850109,126,126,122,123,324836
9284,19850108,129,129,127,127,845098
9285,19850107,129,130,128,129,771895
9286,19850105,129,129,128,128,108496


컬럼은 [일자, 시가, 고가, 저가, 종가, 거래량]으로 구성된 매우 단순한 데이터 셋입니다. 총 9,288개의 row로 구성되어 있네요. 우리는 일자, 시가, 고가, 저가, 거래량을 토대로 미래의 주가인 “종가”를 예측해 보도록 하겠습니다.

날짜 datetime 포맷으로 변환

In [4]:
pd.to_datetime(df_price['일자'], format='%Y%m%d')
# 0      2020-01-07
# 1      2020-01-06
# 2      2020-01-03
# 3      2020-01-02
# 4      2019-12-30

df_price['일자'] = pd.to_datetime(df_price['일자'], format='%Y%m%d')
df_price['연도'] =df_price['일자'].dt.year
df_price['월'] =df_price['일자'].dt.month
df_price['일'] =df_price['일자'].dt.day

In [6]:
df_price

,일자,시가,고가,저가,종가,거래량,연도,월,일
0,2020-01-07,55700,56400,55600,55800,9893846,2020,1,7
1,2020-01-06,54900,55600,54600,55500,10278951,2020,1,6
2,2020-01-03,56000,56600,54900,55500,15422255,2020,1,3
3,2020-01-02,55500,56000,55000,55200,12993228,2020,1,2
4,2019-12-30,56200,56600,55700,55800,8356767,2019,12,30
...,...,...,...,...,...,...,...,...,...
9283,1985-01-09,126,126,122,123,324836,1985,1,9
9284,1985-01-08,129,129,127,127,845098,1985,1,8
9285,1985-01-07,129,130,128,129,771895,1985,1,7
9286,1985-01-05,129,129,128,128,108496,1985,1,5


In [7]:
plt.figure(figsize=(16, 9))
sns.lineplot(y=df['종가'], x=df['일자'])
plt.xlabel('time')
plt.ylabel('price')

NameError: name 'plt' is not defined